In [5]:
import os
import datetime
import time
from dotenv import load_dotenv
import anthropic
from firecrawl import FirecrawlApp
import json

# Load environment variables
load_dotenv()

# Retrieve API keys from environment variables
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
firecrawl_api_key = os.getenv("FIRECRAWL_API_KEY")


In [7]:
# Initialize the FirecrawlApp with your API key
app = FirecrawlApp(api_key=firecrawl_api_key)

# Crawl a website
crawl_url = 'https://dify.ai/'
params = {
    'crawlOptions': {
        'limit': 10
    }
}
crawl_result = app.crawl_url(crawl_url, params=params)

if crawl_result is not None:
    # Convert crawl results to JSON format, excluding 'content' field from each entry
    cleaned_crawl_result = [{k: v for k, v in entry.items() if k != 'content'} for entry in crawl_result]

    # Save the modified results as a text file containing JSON data
    with open('crawl_result.txt', 'w') as file:
        file.write(json.dumps(cleaned_crawl_result, indent=4))
else:
    print("No data returned from crawl.")


In [20]:
import requests
import os
import json

website_dump = open('crawl_result.txt', 'r').read()

anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

headers = {
    "content-type": "application/json",
    "x-api-key": anthropic_api_key,
    "anthropic-version": "2023-06-01",
    "anthropic-beta": "prompt-caching-2024-07-31"
}

data = {
    "model": "claude-3-5-sonnet-20240620",
    "max_tokens": 1024,
    "system": [
        {
            "type": "text",
            "text": "You are an AI assistant tasked with analyzing literary works. Your goal is to provide insightful commentary on themes, characters, and writing style.\n"
        },
        {
            "type": "text",
            "text": website_dump,
            "cache_control": {"type": "ephemeral"}
        }
    ],
    "messages": [
        {
            "role": "user",
            "content": "How can I improve the copy on this website?"
        }
    ]
}

response = requests.post(
    "https://api.anthropic.com/v1/messages",
    headers=headers,
    data=json.dumps(data)
)

print(response.json())



{'id': 'msg_01MLSgCP12zVjSbJhPNBFoCt', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-5-sonnet-20240620', 'content': [{'type': 'text', 'text': 'Based on my analysis of the website content, here are some suggestions to potentially improve the Dify.AI website:\n\n1. Simplify navigation: The top menu has multiple repetitions of "Product", "Pricing", "Docs", and "Blog". Streamline this to avoid confusion and redundancy.\n\n2. Improve mobile responsiveness: Ensure all elements, including images and text, scale properly on different screen sizes for better user experience.\n\n3. Enhance content organization: The page contains a lot of information. Consider organizing it into clearer sections with consistent formatting to improve readability.\n\n4. Optimize loading speed: Large images and numerous elements may slow down the page. Optimize images and consider lazy loading for content further down the page.\n\n5. Add more specific call-to-actions (CTAs): While there are "Get Started